Dependencies

In [ ]:
!pip install gensim==3.7.1

In [ ]:
!pip install imblearn

In [ ]:
!pip install numpy --upgrade

In [ ]:
!pip install keras

In [ ]:
!pip install tensorflow

In [1]:
import pandas as pd 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import ast
from sklearn.model_selection import train_test_split

from sklearn.utils import class_weight
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
C:\Users\Chan Ho Park\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
Using TensorFlow backend.


# Overview of dataset

In [2]:
file_dir = "./data/movies_metadata.csv"
data = pd.read_csv(file_dir)
print(data.columns)
data = data[['genres','overview','tagline']]
print(data.head(5))


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                                            overview  \
0  Led by Woody, Andy's toys live happily in his ...   
1  When siblings Judy and Peter discover an encha...   
2  A family wedding reignites the ancient feud be...   
3  Cheated on, mistreated and steppe

C:\Users\Chan Ho Park\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(data.shape)
print(data.columns)

(45466, 3)
Index(['genres', 'overview', 'tagline'], dtype='object')


In [4]:
def tolist(data):
    dic = ast.literal_eval(data)  
    out = []
    for g in dic:
        out.append(g['name'])
    return out
allg = []
for index,row in data.iterrows():
    dic = ast.literal_eval(row['genres'])  
    for g in dic:
        try:
            allg.index(g['name'])
        except:
            allg.append(g['name'])
data['all'] = data['genres'].apply(tolist)
print("Number of genre is %d \n" %(len(allg)))
print(allg)
data.head(5)


Number of genre is 32 

['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie', 'Carousel Productions', 'Vision View Entertainment', 'Telescene Film Group Productions', 'Aniplex', 'GoHands', 'BROSTA TV', 'Mardock Scramble Production Committee', 'Sentai Filmworks', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel']


,genres,overview,tagline,all
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...",NaN,"[Animation, Comedy, Family]"
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,"[Adventure, Fantasy, Family]"
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,"[Romance, Comedy]"
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...,"[Comedy, Drama, Romance]"
4,"[{'id': 35, 'name': 'Comedy'}]",Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...,[Comedy]


## Information about genre 

### Total list of genre and number

### Identifiying minor labels

In [5]:
def label2vector(genre,allg):
    out = [0]*len(allg)
    dic = ast.literal_eval(genre)  
    for g in dic:
        out[allg.index(g['name'])] = 1
        break
    return out

data['v_genre'] = data['genres'].apply(label2vector, args=(allg,))
print(data[['genres','v_genre']].head(5))

count_g = [0]* len(allg)
for i,row in data['v_genre'].items():
    for j,r in zip(range(len(row)),row):
        if r == 1:
            count_g[j] +=1
print(count_g)


                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                                             v_genre  
0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
[1124, 8820, 524, 1514, 704, 1191, 11966, 4489, 1685, 1665, 2619, 279, 647, 554, 379, 118, 487, 3415, 451, 390, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0]


### Removal of minor label index

In [6]:
rm_list = [ 'Carousel Productions', 'Vision View Entertainment', 'Telescene Film Group Productions','Aniplex', 'GoHands', 'BROSTA TV', 'Mardock Scramble Production Committee', 'Sentai Filmworks', 'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel']
rm_index = []
for i,row in data['v_genre'].items():
    for j,r in zip(range(len(row)),row):
        if r == 1 and j > 19 and j <32:
            rm_index.append(i)
print(rm_index)
data = data.drop(rm_index)
data['v_genre'] = data['v_genre'].apply(lambda x: x[:20])
print(data['v_genre'].iloc[1])

[19730, 29503, 35587]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Cleaning description's stopword

- processing string data through nltk tool and stopword list

In [7]:
from nltk.stem import WordNetLemmatizer

def tag(data,lemma,stop_words):
    filtered = ""
    try:
        data = word_tokenize(data)
        data = nltk.pos_tag(data)
        for word in data:
            if word[0] not in stop_words and len(word[0])>2:
                try: 
                    word[1].index('V')
                    filtered = filtered + " " + lemma.lemmatize(word[0].lower(),'v')
                except:
                    filtered = filtered + " " + lemma.lemmatize(word[0].lower())
    except:
        pass  
    return filtered
def add(row):
    if len(str(row['tagline'])) > 3:
        return str(row['overview'])+" "+str(row['tagline'])
    else:
        return str(row['overview'])
#concatenate overview and tagline
data['description'] = data[['overview','tagline']].apply(add, axis =1)
print("Before:")
print(data.iloc[0]['description'])
print("\n___________________________________________________________________________\n")
#remove stopword
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
data['description'] = data['description'].apply(tag,args=(lemma,stop_words,))
print("After:")
print(data.iloc[0]['description'])


Before:
Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.

___________________________________________________________________________

After:
 lead woody andy toy live happily room andy birthday bring buzz lightyear onto scene afraid lose place andy heart woody plot buzz but circumstance separate buzz woody owner duo eventually learn put aside difference


### Convert description to vector
     - Using pretrained model

In [ ]:
def des2vec(data,model):
    return model.infer_vector(word_tokenize(data))

model= Doc2Vec.load("./models/doc2vec.bin")
data['v_description'] = data['description'].apply(des2vec, args=(model,))


In [8]:
from gensim import models

def wordvec(data,model):
    i = 0.0
    out = 0.0
    for word in str(data).split():
        if word in model.vocab:
            out += model[word]
            i += 1
    if i != 0:
        out = out/i
    else:
        print(data)
    return out
data = data.drop(['genres','tagline'], axis=1)
model2= models.KeyedVectors.load_word2vec_format('./models/GoogleNews.bin',binary=True)
data['w_description'] = data['description'].apply(wordvec, args=(model2,))
data.iloc[1]['w_description']



 remu kasvaa vaatimattomissa oloissa teinipoikana hän jättänyt koulunsa kesken ammatiltaan rikollinen suvun kohtalo toteutumassa odotettavissa tavallinen tarina vankila alkoholismi väkivaltainen kuolema
 ...


 москва город без прошлого без будущего город который прощает ошибок московская богема криминальный бизнес ночная жизнь легкие деньги возбуждение растерянность образуют поверхность этой жизни время очередной перевозки партии нала для нового русского кличке майкл деньги исчезают ситуация осложняется еще больше когда накануне свадьбы невеста майкла влюбляется его друга которого тот подозревает похищении денег герои пытаются заглушить страх перед разрушительной жестокостью жизни алкоголем сексом интрига продолжает разрастаться пока проблема разрешается наиболее универсальным способом пулей затылок…
 ульяна тулина необычный человек взрослом теле душа странного наивного ребенка поступки импульсивные нелепые смешат удивляют раздражают окружающих единственный родной человек который за

array([ 0.05782209,  0.03871881,  0.00344013,  0.08453442,  0.00065249,
        0.01060922,  0.05268787, -0.04557037,  0.05272275,  0.08696493,
       -0.02750796, -0.11166745, -0.02641165, -0.06777664, -0.09281522,
        0.05298287,  0.07704163,  0.10824221,  0.03521583, -0.05674635,
        0.05337234, -0.00189427,  0.10666484,  0.02248201,  0.00968474,
       -0.0568651 , -0.07252975,  0.08453151,  0.09002177, -0.05923752,
       -0.05219232, -0.01393781, -0.04170845,  0.01729693, -0.06512742,
       -0.00980196,  0.00528826,  0.00168573,  0.04685647,  0.09290949,
        0.10045006, -0.04267447,  0.12362526,  0.03561183,  0.00129918,
       -0.07345654, -0.00184195,  0.0200693 ,  0.02873839,  0.04418001,
       -0.06099229,  0.11389596,  0.0183229 , -0.05935669,  0.01588694,
       -0.00556001,  0.00986008, -0.18343608,  0.04042853, -0.04620434,
        0.00703076,  0.04824611, -0.01732527, -0.0471671 , -0.07436553,
       -0.08427066,  0.00455439,  0.01441919, -0.03253428,  0.13

In [ ]:
def similar(text,v):
    out = 0.0
    i2 = 0
    j2 = 0
    for i,j in zip(text,v):
        out = out + i *j
        i2 = i2 + i*i
        j2 = j2 + j*j
    return out, i2,j2
maxx = 0
idd = 0
text = data.iloc[0]['v_description']
for i,v in data['v_description'].items():
    n,i2,j2 = similar(text,v)
    n = n/(i2*j2)**0.5
    if i == 0:
        continue
    elif n < 0.1:
        print("Similarity =",n)
        print(data.iloc[i]['description'])
        print(i)
        print("_____________________________________________________________________")
    elif n > maxx  and n <0.6768:
        maxx = n
        idd =i
print("Similarity =",maxx)
print(data.iloc[idd]['description'])
print(idd)
print("_____________________________________________________________________")

### Remove non-english description

In [9]:
rm = []
for i,v in data['w_description'].items():
    try:
        if v == 0.0:
            rm.append(i) 
    except:
        pass
print(rm)
data = data.drop(rm,axis=0)
print(data.shape)

[4246, 4538, 12769, 13346, 18171, 28408, 29103, 30042, 34244, 34256, 34919, 35314, 36785, 43154, 43533, 44107, 44157, 44859, 45412]
(45444, 5)


### Remove sample with no label

In [10]:
no_genre = []
for i,v in data['v_genre'].items():
    if v == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
        no_genre.append(i)
data = data.drop(no_genre)
print("Number of sample with no label: ",len(no_genre))
data.shape

Number of sample with no label:  2437


(43007, 5)

### Split into train and test data
    - stratified data

In [17]:
from numpy import array
def split_data(data, proportion):
    n = data.shape[0]
    tmp = 0
    x_test = [] 
    y_test = []
    x_train = [] 
    y_train = []
    train_all = []
    
    for index, row in data.iterrows():
        if n* proportion > tmp:
            x_test.append(row['w_description'])
            y_test.append(row['v_genre'])
        else:
            x_train.append(row['w_description'])
            y_train.append(row['v_genre'])
            train_all.append(row['all'])
        tmp+= 1
    x_test = array(x_test)
    y_test = array(y_test)
    x_train = array(x_train)
    y_train = array(y_train)
    return x_test, y_test , x_train, y_train,train_all

def tovec(row,allg):
    out = [0]*20
    for d in row:
        out[allg.index(d)] = 1
    return out

# data['all'] = data['all'].apply(tovec,args=(allg,))

dist = [0] * 20
y_int = []
for j,y in data['v_genre'].items():
    for i,g in zip(range(20),y):
        if g == 1:
            dist[i] += 1
            y_int.append(i)
x_test, y_test , x_train, y_train, test_all = split_data(data,0.15)
print(x_test[1])
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)


[ 0.05782209  0.03871881  0.00344013  0.08453442  0.00065249  0.01060922
  0.05268787 -0.04557037  0.05272275  0.08696493 -0.02750796 -0.11166745
 -0.02641165 -0.06777664 -0.09281522  0.05298287  0.07704163  0.10824221
  0.03521583 -0.05674635  0.05337234 -0.00189427  0.10666484  0.02248201
  0.00968474 -0.0568651  -0.07252975  0.08453151  0.09002177 -0.05923752
 -0.05219232 -0.01393781 -0.04170845  0.01729693 -0.06512742 -0.00980196
  0.00528826  0.00168573  0.04685647  0.09290949  0.10045006 -0.04267447
  0.12362526  0.03561183  0.00129918 -0.07345654 -0.00184195  0.0200693
  0.02873839  0.04418001 -0.06099229  0.11389596  0.0183229  -0.05935669
  0.01588694 -0.00556001  0.00986008 -0.18343608  0.04042853 -0.04620434
  0.00703076  0.04824611 -0.01732527 -0.0471671  -0.07436553 -0.08427066
  0.00455439  0.01441919 -0.03253428  0.13968186  0.09002177  0.01585524
  0.06788126 -0.0071244  -0.07991318 -0.04710751  0.03262039  0.03671101
  0.00841843  0.03296879  0.00231207 -0.02927253  0.

In [ ]:
data.head(5)

### Build Doc2vec Model

In [ ]:
from gensim.test.utils import common_texts

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
j = len(documents)
for i, d in enumerate(data['description']):
    try:
        tagged_data.append(TaggedDocument(words=word_tokenize(d.lower()), tags=[str(i)]))
    except:
        j+=1
        pass

In [ ]:
max_epochs = 100
vec_sizes = [50, 100,200]
alpha = 0.025
for vec_size in vec_sizes:
    model = Doc2Vec(size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1)
    model.build_vocab(tagged_data)
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        model.alpha -= 0.0002
        model.min_alpha = model.alpha
    model.save("doc2vec.model_"+str(vec_size))

## Forward Feeding Neural Network with basic data

### Hyper-parameters

In [12]:
def num_hidden_layer1(num_input, num_output, size):
    # alpha is usually set to be a value to be between 2-10
    alpha = 10
    return [int(size /(num_input + num_output) / alpha)]
def num_hidden_layer2(num_input, num_output,size):
    # when single hidden layer
    return [int(2*((num_output +2)*num_input)**0.5)]
def num_hidden_layer3(num_input, num_output,size):
    # when two hidden layer
    return [int(((num_output+2)*num_input)**0.5 + 2*(num_input/(num_output+2))**0.5), int(num_output*(num_input/(num_output+2))**0.5)]

vec_size = 300
in_len = vec_size # number of input feature
out_len = 20 # number of output label
hidden_layer = num_hidden_layer2(in_len,out_len,data.shape[0])
activation = 'relu'
ep = 100
print(hidden_layer)

[162]


In [13]:
nn = Sequential()
if len(hidden_layer) == 1:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
elif len(hidden_layer) == 2:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
    nn.add(Dense(hidden_layer[1],activation=activation))
elif len(hidden_layer) == 3:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
    nn.add(Dense(hidden_layer[1],activation=activation))
    nn.add(Dense(hidden_layer[2],activation=activation))
nn.add(Dense(out_len, activation='softmax'))
nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
history = nn.fit(x_train, y_train, epochs=ep, batch_size=20, verbose=1, validation_data=(x_test, y_test))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 36555 samples, validate on 6452 samples
Epoch 1/100
36555/36555 [==============================] - 3s 85us/step - loss: 2.4065 - acc: 0.2765 - val_loss: 2.1948 - val_acc: 0.2903
Epoch 2/100
36555/36555 [==============================] - 3s 83us/step - loss: 2.2820 - acc: 0.2917 - val_loss: 2.1499 - val_acc: 0.3007
Epoch 3/100
36555/36555 [==============================] - 3s 85us/step - loss: 2.2248 - acc: 0.3141 - val_loss: 2.0848 - val_acc: 0.3321
Epoch 4/100
36555/36555 [==============================] - 3s 77us/step - loss: 2.1593 - acc: 0.3491 - val_loss: 2.0245 - val_acc: 0.3830
Epoch 5/100
36555/36555 [==============================] - 3s 75us/step - loss: 2.0968 - acc: 0.3755 - val_loss: 1.9721 - val_acc: 0.4011
Epoch 6/100
36555/36555 [==============================] - 3s 85us/step - loss: 2.0448 - acc: 0.3914 - val_loss: 1.9327 - val_acc: 0.4120
Epo

36555/36555 [==============================] - 3s 78us/step - loss: 1.6546 - acc: 0.4807 - val_loss: 1.6221 - val_acc: 0.4709
Epoch 56/100
36555/36555 [==============================] - 3s 77us/step - loss: 1.6526 - acc: 0.4800 - val_loss: 1.6182 - val_acc: 0.4740
Epoch 57/100
36555/36555 [==============================] - 3s 77us/step - loss: 1.6513 - acc: 0.4790 - val_loss: 1.6167 - val_acc: 0.4682
Epoch 58/100
36555/36555 [==============================] - 3s 78us/step - loss: 1.6493 - acc: 0.4818 - val_loss: 1.6166 - val_acc: 0.4735
Epoch 59/100
36555/36555 [==============================] - 3s 82us/step - loss: 1.6478 - acc: 0.4815 - val_loss: 1.6163 - val_acc: 0.4696
Epoch 60/100
36555/36555 [==============================] - 3s 82us/step - loss: 1.6461 - acc: 0.4814 - val_loss: 1.6099 - val_acc: 0.4723
Epoch 61/100
36555/36555 [==============================] - 3s 78us/step - loss: 1.6449 - acc: 0.4815 - val_loss: 1.6210 - val_acc: 0.4712
Epoch 62/100
36555/36555 [==============

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np 
from numpy import array
import keras as K

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(20)
    plt.xticks(tick_marks, allg[:20], rotation=45)
    plt.yticks(tick_marks, allg[:20])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

test_pred = nn.predict(x_test)
conv_y = []
conv_pred = []
for y in y_test:
    conv_y.append(np.argmax(y))
for y in test_pred:
    conv_pred.append(np.argmax(y))
title ="Confusion matrix"
cm = confusion_matrix(conv_y,conv_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(12,10))
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')


## Regularisation

In [ ]:
nn_reg = Sequential()
if len(hidden_layer) == 1:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation,kernel_regularizer=l2(0.1)))
elif len(hidden_layer) == 2:
    nn_reg.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation, kernel_regularizer=l2(0.01)))
    nn_reg.add(Dense(hidden_layer[1],activation=activation))
nn_reg.add(Dense(out_len, activation='softmax'))
nn_reg.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
history_reg = nn_reg.fit(x_train, y_train, epochs=ep, batch_size=20, verbose=1, validation_data=(x_test, y_test))


## Class weight / Label penalty 

If you are talking about the regular case, where your network produces only one output, then your assumption is correct.
In order to force your algorithm to treat every instance of class 1 as 50 instances of class 0 you have to:
class_weight = {0: 1.,
                1: 50.,
                2: 2.}

In [ ]:
import math
from sklearn.utils.class_weight import compute_class_weight
# method1 
# dist = [0] * 20
# for y in y_train:
#     for i,g in zip(range(20),y):
#         if g == 1:
#             dist[i] += 1
# s = 0
# for d in dist:
#     s += d
# weight_dic = {}

#method2 
dist = [0] * 20
y_train_int = []
# convert vector label to int label for SMOTE
for y in y_train:
    for i,g in zip(range(20),y):
        if g == 1:
            dist[i] += 1
            y_train_int.append(i)
weights = compute_class_weight('balanced', range(20), y_train_int)
print(weights)


In [ ]:
weight_dic = {}
for i,w in enumerate(weights):
    weight_dic[i] = w
print("Class weight is : \n",weight_dic)

nn_weight = Sequential()
if len(hidden_layer) == 1:
    nn_weight.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
elif len(hidden_layer) == 2:
    nn_weight.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
    nn_weight.add(Dense(hidden_layer[1],activation=activation))
nn_weight.add(Dense(out_len, activation='softmax'))
nn_weight.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
history_class = nn_weight.fit(x_train, y_train, epochs=100, batch_size=20, verbose=1, validation_data=(x_test, y_test), class_weight=weight_dic)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np 
from numpy import array
import keras as K

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(20)
    plt.xticks(tick_marks, allg[:20], rotation=45)
    plt.yticks(tick_marks, allg[:20])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

test_pred = nn_weight.predict(x_test)
conv_y = []
conv_pred = []
for y in y_test:
    conv_y.append(np.argmax(y))
for y in test_pred:
    conv_pred.append(np.argmax(y))
title ="Confusion matrix"
cm = confusion_matrix(conv_y,conv_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(12,10))
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')


## Oversampling/ SMOTE data 

### Distribution of Genres in Dataset / Resampling (SMOTE)

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN

dist = [0] * 20
y_train_int = []
# convert vector label to int label for SMOTE
for y in y_train:
    for i,g in zip(range(20),y):
        if g == 1:
            dist[i] += 1
            y_train_int.append(i)

# check distribution of labels
s= 0
for d in dist:
    s += d
for d,g in zip(dist,allg):
    print("%.2f "%(d/s) + g)

x_resampled, y_resampled = SMOTE().fit_resample(x_train, y_train_int)

# check distribution after oversampling
print("After SMOTE \n")
dist = [0] * 20
for y in y_resampled:
    dist[y] += 1
s = 0
for d in dist:
    s += d
for d,g in zip(dist,allg):
    print("%.2f "%(d/s) + g)

# X_resampled, y_resampled = ADASYN().fit_resample(x_train, y)
#back to one hot encoding
y_resampled_conv = []
for y in y_resampled:
    add = [0] * 20
    add[y] = 1
    y_resampled_conv.append(add)
y_resampled_conv = array(y_resampled_conv)
print(y_resampled_conv[1])
nn = Sequential()
if len(hidden_layer) == 1:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
elif len(hidden_layer) == 2:
    nn.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
    nn.add(Dense(hidden_layer[1],activation=activation))
nn.add(Dense(out_len, activation='softmax'))
nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history_re = nn.fit(x_resampled, y_resampled_conv, epochs=ep, batch_size=20, verbose=1, validation_data=(x_test, y_test))


from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np 
from numpy import array
import keras as K

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(20)
    plt.xticks(tick_marks, allg[:20], rotation=45)
    plt.yticks(tick_marks, allg[:20])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

test_pred = nn.predict(x_test)
conv_y = []
conv_pred = []
for y in y_test:
    conv_y.append(np.argmax(y))
for y in test_pred:
    conv_pred.append(np.argmax(y))
title ="Confusion matrix"
cm = confusion_matrix(conv_y,conv_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(12,10))
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')


In [ ]:
file_dir = "./data/movies_metadata.csv"
data2 = pd.read_csv(file_dir)
data2 = data2['genres']

rm = []
all_label = []
for i,row in data2.items():
    dic = ast.literal_eval(row)  
    dic = ast.literal_eval(row)  
    out = []
    for g in dic:
        out.append(g['name'])
    if len(out) == 0:
        rm.append(i)
    all_label.append(out)
data2 = data2.drop([19730, 29503, 35587, 4246, 4538, 12769, 13346, 28408, 29103, 30042, 34244, 34256, 36785, 43154, 44107, 44157, 45412])
data2 = data2.drop(rm)
data2.shape
data2.head(5)
print(all_label[1:4])

### Combining two models

In [18]:
def in_max(l):
    m = 0
    out = 0
    for i,j in zip(range(20),l):
        if m < j:
            m = j
            out = i
    return out
def score(true,final):
    correct = 0
    for i,j in zip(true,final):
        try:
            i.index(j)
            correct +=1
        except:
            continue
    print(correct/len(true))

pred_weight = nn.predict(x_train)
pred_nn = nn.predict(x_train)
final = []
for r1,r2 in zip(pred_weight,pred_nn):
    add = []
    for v1,v2 in zip(r1,r2):
        add.append((v1+v2)/2)
    final.append(in_max(add))
print("First five prediction :",[allg[i] for i in final[0:5]])
final = array(final)
true = []
#print(test_all[1:5])
for i in test_all:
    add =[]
    for p,q in zip(range(20),i):
        if q != 0:
            add.append(p)
    true.append(add)
print("True genre of first five\n"+
'''[Animation, Comedy, Family] 
[Adventure, Fantasy, Family]
[Romance, Comedy]
[Comedy, Drama, Romance]
[Comedy]
''')
score(true,final)


First five prediction : ['Action', 'Horror', 'Comedy', 'Drama', 'Comedy']
True genre of first five
[Animation, Comedy, Family] 
[Adventure, Fantasy, Family]
[Romance, Comedy]
[Comedy, Drama, Romance]
[Comedy]

0.15316646149637533


### Undersampling

In [ ]:
from imblearn.keras import balanced_batch_generator
from imblearn.under_sampling import NearMiss


training_generator, steps_per_epoch = balanced_batch_generator(
                                        x_train, y_train, sampler=NearMiss(), batch_size=10, random_state=42)

nn_under = Sequential()
if len(hidden_layer) == 1:
    nn_under.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
elif len(hidden_layer) == 2:
    nn_under.add(Dense(hidden_layer[0],input_dim=in_len,activation=activation))
    nn_under.add(Dense(hidden_layer[1],activation=activation))
nn_under.add(Dense(out_len, activation='softmax'))
nn_under.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
under_hist = nn_under.fit_generator(generator=training_generator,
                                        steps_per_epoch=steps_per_epoch,
                                        epochs=150, verbose=1)
print(nn_under.evaluate(x_test,y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
import numpy as np 
from numpy import array
import keras as K

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(20)
    plt.xticks(tick_marks, allg[:20], rotation=45)
    plt.yticks(tick_marks, allg[:20])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

test_pred = nn_under.predict(x_test)
conv_y = []
conv_pred = []
for y in y_test:
    conv_y.append(np.argmax(y))
for y in test_pred:
    conv_pred.append(np.argmax(y))
title ="Confusion matrix"
cm = confusion_matrix(conv_y,conv_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(12,10))
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')


### Log loss plot

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# combine two models

def final_pred(nn_model, cnn_model,x_test,y_test):
    nn_pred = nn_model.predict(x_test)
    cnn_pred = cnn_model.predict(x_test)
    pred = []
    for i,j in zip(nn_pred,cnn_pred):
        sample_pred = []
        for p1,p2 in zip(i,j):
            sample_pred.append((i+j)/2)
        sample_pred[np.argmax()]